In [1]:
import os

In [37]:
import psycopg2
from psycopg2 import pool

Récupération du mot de passe (Variable d'environnement)

In [3]:
PASSWORD = os.environ.get('PASSWORD')

Initialisation des variables de connexion à Azure

In [74]:
dbname='postgres' 
user='azure_admin@sprengerdemo' 
host='sprengerdemo.postgres.database.azure.com' 
password=PASSWORD 
port='5432' 
sslmode='require'


# 4) Connection à Azure

In [75]:
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port, sslmode=sslmode)

# 5) Création de tables et des schémas

In [91]:
cursor = conn.cursor()

On va créer une table `Alert` pour stocker les alertes dans la base de données pour tracker les cryptos et envoyer un mail de notification si le prix d'une monnaie est au-dessus ou en dessous d'une valeur prédéfinie par un utilisateur.

Une alerte est composée d'un `mail` de contact, d'une crypto monnaies `currency`, d'un prix `price` la valeur de seuil, ainsi qu'une méthode `method` contenant la valeur `below` ou `above`.

In [90]:
create_table_query = """
    CREATE TABLE IF NOT EXISTS alert (
        id SERIAL PRIMARY KEY,
        mail VARCHAR(255),
        currency VARCHAR(255),
        price FLOAT,
        method VARCHAR(255)
    )
"""

cursor.execute(create_table_query)
conn.commit()

# 6) Exécution de requêtes SQL à distance pour récupérer des données et alimenter la base de données de nouvelle valeur

On va alimenter la base de données de nouvelle et les push sur Azure

In [93]:
insert_data_query = """
    INSERT INTO alert (mail, currency, price, method)
    VALUES ('test@gmail.com', 'BTC', 1000, 'above')
"""

cursor.execute(insert_data_query)
conn.commit()

In [94]:
insert_data_query = """
    INSERT INTO alert (mail, currency, price, method)
    VALUES ('test@gmail.com', 'BTC', 3000, 'above')
"""

cursor.execute(insert_data_query)
conn.commit()

In [101]:
insert_data_query = """
    INSERT INTO alert (mail, currency, price, method)
    VALUES ('test@gmail.com', 'ETH', 5000, 'below')
"""

cursor.execute(insert_data_query)
conn.commit()

On récupère toute les données de la base de données Azure

In [97]:
select_all_query = "SELECT * FROM alert"
cursor.execute(select_all_query)
rows = cursor.fetchall()
for row in rows:
    print(row)

(1, 'test@gmail.com', 'BTC', 1000.0, 'above')
(2, 'test@gmail.com', 'BTC', 1000.0, 'above')
(3, 'test@gmail.com', 'BTC', 3000.0, 'above')
(4, 'test@gmail.com', 'ETH', 1000.0, 'below')


On récupère uniquement les alertes de BITCOIN dans la base de données d'Azure

In [103]:
select_query = "SELECT * FROM alert WHERE currency = 'BTC' AND method = 'above'"
cursor.execute(select_query)

rows = cursor.fetchall()
for row in rows:
    print(row)

(1, 'test@gmail.com', 'BTC', 1000.0, 'above')
(2, 'test@gmail.com', 'BTC', 1000.0, 'above')
(3, 'test@gmail.com', 'BTC', 3000.0, 'above')


### 7) Expérimentation des opérations de lecture (SELECT), d’insertion (INSERT), de mise à jour (UPDATE) et de suppression (DELETE) de données dans la base de données

On va mettre à jour une alerte déja présente dans la base de données

In [104]:
update_query = """
    UPDATE alert
    SET price = 1500
    WHERE id = 1
"""

cursor.execute(update_query)
conn.commit()

On remarque bien que la valeur a bien été modifiée

In [105]:
select_all_query = "SELECT * FROM alert"
cursor.execute(select_all_query)
rows = cursor.fetchall()
for row in rows:
    print(row)

(2, 'test@gmail.com', 'BTC', 1000.0, 'above')
(3, 'test@gmail.com', 'BTC', 3000.0, 'above')
(4, 'test@gmail.com', 'ETH', 1000.0, 'below')
(5, 'test@gmail.com', 'BTC', 1000.0, 'below')
(6, 'test@gmail.com', 'BTC', 5000.0, 'below')
(7, 'test@gmail.com', 'ETH', 5000.0, 'below')
(1, 'test@gmail.com', 'BTC', 1500.0, 'above')


On va rajouter un faux compte dans la base données.

In [110]:
insert_data_query = """
    INSERT INTO alert (mail, currency, price, method)
    VALUES ('fake_account@gmail.com', 'ETH', 5500, 'below')
"""

cursor.execute(insert_data_query)
conn.commit()

In [112]:
select_all_query = "SELECT * FROM alert"
cursor.execute(select_all_query)
rows = cursor.fetchall()
for row in rows:
    print(row)

(2, 'test@gmail.com', 'BTC', 1000.0, 'above')
(3, 'test@gmail.com', 'BTC', 3000.0, 'above')
(4, 'test@gmail.com', 'ETH', 1000.0, 'below')
(5, 'test@gmail.com', 'BTC', 1000.0, 'below')
(6, 'test@gmail.com', 'BTC', 5000.0, 'below')
(7, 'test@gmail.com', 'ETH', 5000.0, 'below')
(1, 'test@gmail.com', 'BTC', 1500.0, 'above')
(8, 'fake_account@gmail.com', 'ETH', 5000.0, 'below')
(9, 'fake_account@gmail.com', 'ETH', 5500.0, 'below')
(10, 'fake_account@gmail.com', 'ETH', 5500.0, 'below')
(11, 'fake_account@gmail.com', 'ETH', 5500.0, 'below')
(12, 'fake_account@gmail.com', 'ETH', 5500.0, 'below')


On va supprimer le faux compte de notre base de données

In [113]:
delete_query = """
    DELETE FROM alert
    WHERE mail = 'fake_account@gmail.com'
"""

cursor.execute(delete_query)
conn.commit()

On a bien notre faux compte qui a été supprimée.

In [114]:
select_all_query = "SELECT * FROM alert"
cursor.execute(select_all_query)
rows = cursor.fetchall()
for row in rows:
    print(row)

(2, 'test@gmail.com', 'BTC', 1000.0, 'above')
(3, 'test@gmail.com', 'BTC', 3000.0, 'above')
(4, 'test@gmail.com', 'ETH', 1000.0, 'below')
(5, 'test@gmail.com', 'BTC', 1000.0, 'below')
(6, 'test@gmail.com', 'BTC', 5000.0, 'below')
(7, 'test@gmail.com', 'ETH', 5000.0, 'below')
(1, 'test@gmail.com', 'BTC', 1500.0, 'above')


In [115]:
cursor.close()
conn.close()